In [1]:
import pandas as pd
import numpy as np

In [2]:
reserve_tb = pd.read_csv('./data/reserve.csv', encoding='UTF-8')

In [3]:
customer_tb = pd.read_csv('./data/customer.csv', encoding='UTF-8')

In [4]:
hotel_tb = pd.read_csv('./data/hotel.csv', encoding='UTF-8')

In [5]:
month_mst = pd.read_csv('./data/month_mst.csv', encoding='UTF-8')

## 13. 연습 문제
### 13.3. 예측 모델링 전처리
#### 예측 모델링을 위한 전처리

In [6]:
reserve_tb = reserve_tb.copy().query('"2016-04-01" <= checkin_date <= "2017-03-31"')

In [8]:
yearmonth_mst = month_mst \
    .query('"2016-04-01" <= month_first_day <= "2017-03-01"')
yearmonth_mst.loc[:, 'yearmonth'] = pd.to_datetime(
    yearmonth_mst['month_first_day'],
    format='%Y-%m-%d'
).apply(lambda x: x.strftime('%Y%m'))

customer_tb.loc[:,'join_key'] = 0
yearmonth_mst.loc[:,'join_key'] = 0

customer_mst = pd.merge(
    customer_tb[['customer_id', 'join_key']],
    yearmonth_mst[['join_key', 'yearmonth']],
    on='join_key'
)
customer_mst = customer_mst[['customer_id', 'yearmonth']]
customer_mst

    year_num  month_num month_first_day month_last_day yearmonth
3       2016          4      2016-04-01     2016-04-30    201604
4       2016          5      2016-05-01     2016-05-31    201605
5       2016          6      2016-06-01     2016-06-30    201606
6       2016          7      2016-07-01     2016-07-31    201607
7       2016          8      2016-08-01     2016-08-31    201608
8       2016          9      2016-09-01     2016-09-30    201609
9       2016         10      2016-10-01     2016-10-31    201610
10      2016         11      2016-11-01     2016-11-30    201611
11      2016         12      2016-12-01     2016-12-31    201612
12      2017          1      2017-01-01     2017-01-31    201701
13      2017          2      2017-02-01     2017-02-28    201702
14      2017          3      2017-03-01     2017-03-31    201703
  customer_id  join_key
0         c_1         0
1         c_2         0
2         c_3         0
3         c_4         0
4         c_5         0
   join_key

,customer_id,yearmonth
0,c_1,201604
1,c_1,201605
2,c_1,201606
3,c_1,201607
4,c_1,201608
...,...,...
11995,c_1000,201611
11996,c_1000,201612
11997,c_1000,201701
11998,c_1000,201702


In [15]:
reserve_tb['checkin_datetime'] = pd.to_datetime(
    reserve_tb['checkin_date'] + reserve_tb['checkin_time'],
    format='%Y-%m-%d%H:%M:%S'
)
reserve_tb['yearmonth'] = reserve_tb['checkin_datetime'].apply(
    lambda x: x.strftime("%Y%m")
)
reserve_tb.drop(['checkin_datetime'], axis=1)
reserve_tb['yearmonth']

gp_tb = reserve_tb \
    .groupby(['customer_id', 'yearmonth']) \
    .agg({
        'reserve_id': 'count',
    }) \
    .reset_index()
gp_tb.columns = ['customer_id', 'yearmonth', 'rsv_cnt']

flg_tb = pd.merge(
    customer_mst,
    gp_tb,
    on=['customer_id', 'yearmonth'],
    how='left'
)
flg_tb.loc[:, 'flg'] = flg_tb['rsv_cnt'].apply(
    lambda x: x == 1.0
)
flg_tb

,customer_id,yearmonth,rsv_cnt,flg
0,c_1,201604,NaN,False
1,c_1,201605,NaN,False
2,c_1,201606,NaN,False
3,c_1,201607,1.0,True
4,c_1,201608,NaN,False
...,...,...,...,...
11995,c_1000,201611,NaN,False
11996,c_1000,201612,NaN,False
11997,c_1000,201701,NaN,False
11998,c_1000,201702,NaN,False


In [16]:
# 분할